# The N-body problem

### Imports

In [ ]:
# Magic function: enables interactive plot
%matplotlib widget
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import plotly.express as px


### Initial conditions

In [ ]:
# body mA initial conditions
mA = 1e26  # mass (kg)
rA0 = np.array([1E3, 0, 0])  # initial position (km)
vA0 = np.array([10, -20, 10])  # initial velocity (km/s)

# body mB initial conditions
mB = 1e26  # mass (kg)
rB0 = np.array([-1E3, 0, 0])  # initial position (km)
vB0 = np.array([10, 40, 10])  # initial velocity (km/s)

In [ ]:
y0 = np.stack([np.stack([rA0, vA0]), np.stack([rB0, vB0])])

In [ ]:
y0.shape

In [ ]:
G = 6.67259e-20  # Gravitational constant (km**3/kg/s**2)

### The derivatives


In [ ]:
def two_body_eqm_derivatives(_y, t, _G, _mA, _mB):
    """
    derivatives of the equations of motion describing the two-body system
    t is unused, but we keep it for consistency with scipy requirement
    """
    rA = _y[0, 0, :]
    rB = _y[1, 0, :]

    vA = _y[0, 1, :]
    vB = _y[1, 1, :]

    # magnitude of position vector from rA to rB
    distance = np.linalg.norm(rB - rA)

    # accelerations
    aA = _G * _mA * ((rB - rA) / np.power(distance, 3))
    aB = _G * _mB * ((rA - rB) / np.power(distance, 3))

    derivatives = np.stack([np.stack([vA, aA]), np.stack([vB, aB])])

    return derivatives


### Forward time evolution

In [ ]:
dt = 0.001  # time step (s)
tf = 1E2  # end of simulation (s)

In [ ]:
def evolve(y0, tf, dt, method, params):
    history = []
    yn = y0
    t_axis = np.arange(0, tf, dt)
    for tn in t_axis:
        yn = evolve_one_step(yn, tn, dt, method, params)
        history.append(yn.copy())

    history = np.stack(history, axis=-1)
    return history

### Euler's method

In [ ]:
def evolve_one_step(yn, tn, dt, method, params):
    if method == "euler":
        f = two_body_eqm_derivatives(yn, tn, *params)
        yn += f * dt
    elif method == "rk4":
        f1 = two_body_eqm_derivatives(yn, tn, *params)
        f2 = two_body_eqm_derivatives(yn + f1 * dt / 2, tn + dt / 2, *params)
        f3 = two_body_eqm_derivatives(yn + f2 * dt / 2, tn + dt / 2, *params)
        f4 = two_body_eqm_derivatives(yn + f3 * dt, tn + dt, *params)
        yn += (f1 + 2 * f2 + 2 * f3 + f4) * dt / 6

    return yn

### Running the simulation

In [ ]:
history = evolve(y0, tf, dt, "euler", params=(G, mA, mB))

In [ ]:
# if we want to compare with odeint from scipy
#from scipy.integrate import odeint
#history = odeint(two_body_eqm_derivatives, y0, np.arange(0, tf, dt), args=(G, mA, mB))

In [ ]:
history.shape

### Plotting the outcomes

In [ ]:
# Trajectories
xA = history[0, 0, 0, :]
yA = history[0, 0, 1, :]
zA = history[0, 0, 2, :]

xB = history[1, 0, 0, :]
yB = history[1, 0, 1, :]
zB = history[1, 0, 2, :]

In [ ]:
plt.style.use('dark_background')

fig = plt.figure()
ax = plt.axes(projection='3d')


def animate(frame_num):
    ax.clear()
    ax.plot3D(xA[:frame_num], yA[:frame_num], zA[:frame_num], c='blue')
    ax.scatter(xA[frame_num], yA[frame_num], zA[frame_num], c='blue', marker='o')

    ax.plot3D(xB[:frame_num], yB[:frame_num], zB[:frame_num], c='orange')
    ax.scatter(xB[frame_num], yB[frame_num], zB[frame_num], c='orange', marker='o')

    xm = np.min(np.concatenate([xA, xB]))
    xM = np.max(np.concatenate([xA, xB]))
    ym = np.min(np.concatenate([yA, yB]))
    yM = np.max(np.concatenate([yA, yB]))
    zm = np.min(np.concatenate([zA, zB]))
    zM = np.max(np.concatenate([zA, zB]))

    ax.set(xlim3d=(xm, xM), xlabel='X')
    ax.set(ylim3d=(ym, yM), ylabel='Y')
    ax.set(zlim3d=(zm, zM), zlabel='Z')


anim = FuncAnimation(fig, animate, frames=len(xA), interval=10, repeat=False)
plt.show()